In [1]:
from corpus_characterizer import generator_chunker
import numpy as np
import os
# from sklearn.metrics import mean_squared_error,mean_absolute_error, median_absolute_error, mean_squared_log_error, explained_variance_score, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, explained_variance_score, \
    r2_score
import pandas as pd
from scipy.stats import describe, kurtosistest, skewtest, normaltest
from Conv1D_LSTM_Ensemble import pair_generator_1dconv_lstm_bagged
from Conv1D_ActivationSearch_BigLoop import pair_generator_1dconv_lstm #NOT BAGGED
from AuxRegressor import create_testing_set, create_training_set
import matplotlib.pyplot as plt 

# @@@@@@@@@@@@@@ RELATIVE PATHS @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Base_Path = "./"
image_path = "./images/"
train_path = "./train/"
test_path = "./test/"
analysis_path = "./analysis/"
chunker_path = analysis_path + "chunker/"
preds_path = analysis_path + "preds/"

#create the data-pair filenames (using zip), use the helper methods
train_set_filenames = create_training_set()
test_set_filenames = create_testing_set()
print(train_set_filenames)
print(test_set_filenames)


Using Theano backend.


after shuffling: [('sequence_1d_125_6.npy', 'sequence_1d_125_6_label_.npy'), ('sequence_2b_224_5_fv1b.npy', 'sequence_2b_224_5_label_fv1b.npy'), ('sequence_1d_131_2.npy', 'sequence_1d_131_2_label_.npy'), ('sequence_1l_63_4.npy', 'sequence_1l_63_4_label_.npy'), ('sequence_1b_106_7_fv1b.npy', 'sequence_1b_106_7_label_fv1b.npy'), ('sequence_1d_140_1.npy', 'sequence_1d_140_1_label_.npy'), ('sequence_1b_75_6_fv1b.npy', 'sequence_1b_75_6_label_fv1b.npy'), ('sequence_1d_152_3.npy', 'sequence_1d_152_3_label_.npy'), ('sequence_1d_141_2.npy', 'sequence_1d_141_2_label_.npy'), ('sequence_1d_101_2.npy', 'sequence_1d_101_2_label_.npy'), ('sequence_1a_72_3.npy', 'sequence_1a_72_3_label_.npy'), ('sequence_1c_0_1_fv1b.npy', 'sequence_1c_0_1_label_fv1b.npy'), ('sequence_1d_100_1.npy', 'sequence_1d_100_1_label_.npy'), ('sequence_2c_272_3_fv1b.npy', 'sequence_2c_272_3_label_fv1b.npy'), ('sequence_1d_127_8.npy', 'sequence_1d_127_8_label_.npy'), ('sequence_1a_40_1.npy', 'sequence_1a_40_1_label_.npy'), ('seq

In [2]:
i = 0 
unique_load_sequences = [] #fill these with the 4-pairs as a numpy array
unique_load_sequences_aslists = []
sequences_4pairs_list = []
sequences_not4pairs_list = []
filename_loadseq_dict_seqidkey = {} #key is the sequence ID, value is its position in the list. 
four_pairs_condition = False
print("train_set size: ",len(train_set_filenames))
for files in train_set_filenames:
    #print("filename", files)
    i += 1
    data_load_path = train_path + '/data/' + files[0]
    label_load_path = train_path + '/label/' + files[1]
    # print("data/label load path: {} \n {}".format(data_load_path,label_load_path))
    train_array = np.load(data_load_path)
    train_label_array = np.load(label_load_path)[:, 1:]
    
    #check if the repeating unit is actually 4 tuples. 
    #four_pair_condition = (train_array[0:5,-2] == train_array[4:8,-2]) and (train_array[0:5,-1] == train_array[4:8,-1]) error. 
    #numpy version 
    four_pair_condition = np.array_equal(train_array[0:4,-2:],train_array[4:8,-2:]) 

    
    if four_pair_condition == True:
        #print(str(files[0])," is in.")
        load_sequence = train_array[0:4,-2:]
        #if len(unique_load_sequences) != 0 :
        array_equality = False
        if len(unique_load_sequences) == 0:
            unique_load_sequences.append(load_sequence)
            #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence)) #need to be flattened into a list to make it hashable for the list-operators to work
            unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        if len(unique_load_sequences) != 0:
            for load_seq in unique_load_sequences:
                if np.array_equal(load_seq,load_sequence) == True:
                    #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                    array_equality = True
            if array_equality == False:
                unique_load_sequences.append(load_sequence) #the load types
                unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
            filename_loadseq_dict_seqidkey[str(files[0])[:-4]] = unique_load_sequences_aslists.index(str(np.ndarray.flatten(load_sequence)))
        sequences_4pairs_list.append(str(files[0]))
    if four_pair_condition == False:
        sequences_not4pairs_list.append(str(files[0]))
        
    
    #print("train_array's last two columns",train_array[0:20,-2:])
#unique_load_sequences = set(unique_load_sequences)
#print("there are", len(unique_load_sequences), " load sequences")
#print("the unique load sequences are:", unique_load_sequences)
print("sequences with 4-pair repeating units: ",len(sequences_4pairs_list))
print("sequences WITHOUT 4-pair repeating units: ",sequences_not4pairs_list)

('train_set size: ', 440)
('sequences with 4-pair repeating units: ', 320)
('sequences WITHOUT 4-pair repeating units: ', ['sequence_1l_63_4.npy', 'sequence_1l_31_2.npy', 'sequence_1l_37_8.npy', 'sequence_1k_207_8.npy', 'sequence_1k_176_7.npy', 'sequence_1k_161_2.npy', 'sequence_1l_7_8.npy', 'sequence_1l_52_3.npy', 'sequence_1l_65_6.npy', 'sequence_1l_30_1.npy', 'sequence_1l_66_7.npy', 'sequence_1k_201_2.npy', 'sequence_1k_215_6.npy', 'sequence_1k_235_6.npy', 'sequence_1l_4_5.npy', 'sequence_1k_212_3.npy', 'sequence_1k_210_1.npy', 'sequence_1k_175_6.npy', 'sequence_1k_170_1.npy', 'sequence_1k_163_4.npy', 'sequence_1k_162_3.npy', 'sequence_1l_76_7.npy', 'sequence_1l_62_3.npy', 'sequence_1l_53_4.npy', 'sequence_1k_186_7.npy', 'sequence_1k_166_7.npy', 'sequence_1l_35_6.npy', 'sequence_1l_51_2.npy', 'sequence_1k_222_3.npy', 'sequence_1l_12_3.npy', 'sequence_1k_223_4.npy', 'sequence_1k_234_5.npy', 'sequence_1k_225_6.npy', 'sequence_1l_3_4.npy', 'sequence_1l_44_5.npy', 'sequence_1l_26_7.npy'

In [3]:
twelve_pair_condition = np.array_equal(train_array[0:12,-2:],train_array[12:24,-2:]) 
sequences_12pairs_list = []
sequences_not4_not12_pairs_list = []

#gotta run the 4-pair filter first to make sure the 12-pair subset list is ok. Hence the separate loop. 
for unusual_seqs in sequences_not4pairs_list: #12 repeating units. 
    data_load_path = train_path + '/data/' + unusual_seqs
    train_array = np.load(data_load_path)
    
    if twelve_pair_condition == True:
        #print(str(unusual_seqs)," is in.")
        load_sequence = train_array[0:12,-2:]
        #if len(unique_load_sequences) != 0 :
        array_equality = False
        if len(unique_load_sequences) == 0:
            unique_load_sequences.append(load_sequence)
            #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
            unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        if len(unique_load_sequences) != 0:
            for load_seq in unique_load_sequences:
                if np.array_equal(load_seq,load_sequence) == True:
                    #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                    array_equality = True
            if array_equality == False:
                unique_load_sequences.append(load_sequence) #the load types
                #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
                unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
                                        
        sequences_12pairs_list.append(str(unusual_seqs))
        filename_loadseq_dict_seqidkey[str(unusual_seqs)[:-4]] = unique_load_sequences_aslists.index(str(np.ndarray.flatten(load_sequence)))
    if twelve_pair_condition == False:
        sequences_not4_not12_pairs_list.append(str(unusual_seqs))

In [4]:
print("There are ",len(sequences_12pairs_list), " sequences with 12-pairs")        
print("There are ", len(sequences_not4_not12_pairs_list), "sequences that have neither 4 nor 12 repeating units")
print("seqs that aren't 4 or 12: ",sequences_not4_not12_pairs_list)
print("there are ",len(unique_load_sequences), " different load sequences in the ", len(train_set_filenames), "sequences in the training set")
#print("unique load seqs", unique_load_sequences)

('There are ', 0, ' sequences with 12-pairs')
('There are ', 120, 'sequences that have neither 4 nor 12 repeating units')
("seqs that aren't 4 or 12: ", ['sequence_1l_63_4.npy', 'sequence_1l_31_2.npy', 'sequence_1l_37_8.npy', 'sequence_1k_207_8.npy', 'sequence_1k_176_7.npy', 'sequence_1k_161_2.npy', 'sequence_1l_7_8.npy', 'sequence_1l_52_3.npy', 'sequence_1l_65_6.npy', 'sequence_1l_30_1.npy', 'sequence_1l_66_7.npy', 'sequence_1k_201_2.npy', 'sequence_1k_215_6.npy', 'sequence_1k_235_6.npy', 'sequence_1l_4_5.npy', 'sequence_1k_212_3.npy', 'sequence_1k_210_1.npy', 'sequence_1k_175_6.npy', 'sequence_1k_170_1.npy', 'sequence_1k_163_4.npy', 'sequence_1k_162_3.npy', 'sequence_1l_76_7.npy', 'sequence_1l_62_3.npy', 'sequence_1l_53_4.npy', 'sequence_1k_186_7.npy', 'sequence_1k_166_7.npy', 'sequence_1l_35_6.npy', 'sequence_1l_51_2.npy', 'sequence_1k_222_3.npy', 'sequence_1l_12_3.npy', 'sequence_1k_223_4.npy', 'sequence_1k_234_5.npy', 'sequence_1k_225_6.npy', 'sequence_1l_3_4.npy', 'sequence_1l_44

In [5]:
#just query this. e.g. print dict[sequence id] 
print("the dict with seq id as key: ", filename_loadseq_dict_seqidkey)

('the dict with seq id as key: ', {'sequence_1a_24_5': 3, 'sequence_1c_34_5_fv1b': 0, 'sequence_2c_267_8_fv1b': 13, 'sequence_2b_174_5_fv1b': 7, 'sequence_1c_20_1_fv1b': 8, 'sequence_1a_64_5': 2, 'sequence_1a_57_8': 1, 'sequence_2b_187_8_fv1b': 5, 'sequence_1a_30_1': 8, 'sequence_1d_151_2': 4, 'sequence_2b_215_6_fv1b': 0, 'sequence_1a_5_6': 7, 'sequence_1a_55_6': 1, 'sequence_1b_67_8_fv1b': 5, 'sequence_2c_247_8_fv1b': 10, 'sequence_2b_186_7_fv1b': 5, 'sequence_1b_105_6_fv1b': 1, 'sequence_2b_224_5_fv1b': 1, 'sequence_1d_91_2': 5, 'sequence_1b_70_1_fv1b': 3, 'sequence_1c_15_6_fv1b': 3, 'sequence_1c_47_8_fv1b': 1, 'sequence_2c_241_2_fv1b': 10, 'sequence_2c_233_4_fv1b': 9, 'sequence_2b_176_7_fv1b': 7, 'sequence_1d_87_8': 7, 'sequence_2b_175_6_fv1b': 7, 'sequence_1c_31_2_fv1b': 0, 'sequence_2c_264_5_fv1b': 13, 'sequence_1d_85_6': 7, 'sequence_1b_73_4_fv1b': 3, 'sequence_2b_191_2_fv1b': 3, 'sequence_1a_61_2': 2, 'sequence_2c_253_4_fv1b': 12, 'sequence_2b_171_2_fv1b': 7, 'sequence_1c_0_1_fv

In [6]:
#make the reverse. load-sequence as key, sequence ID (as lists) as values. 
#dict[key].append(value as list)
filename_loadseq_dict_loadlistposkey = {}

for key,value in filename_loadseq_dict_seqidkey.iteritems():
    print("key",key,"value",value)
    load_sequence_aslist = str(unique_load_sequences_aslists[value])
    #this arrangement below is necessary because dict has to have a list as its value. 
    #casting the list-cast ndarray as a string means.. parsing it back to numpy requires some creativity. 
    if filename_loadseq_dict_loadlistposkey.get(str(load_sequence_aslist)) != None:
        filename_loadseq_dict_loadlistposkey[str(load_sequence_aslist)].append(key)
    if filename_loadseq_dict_loadlistposkey.get(str(load_sequence_aslist)) == None: 
        content_list = []
        content_list.append(key)
        filename_loadseq_dict_loadlistposkey[str(load_sequence_aslist)] = content_list

        
#print(filename_loadseq_dict_loadlistposkey)
# fl_llpk_df = pd.DataFrame.from_dict(filename_loadseq_dict_loadlistposkey,orient='index')
# fl_sik_df = pd.DataFrame.from_dict(filename_loadseq_dict_seqidkey,orient='index')
# fl_llpk_df.to_csv(analysis_path + "file_loadseq_loadlistkey_TRAIN.csv")
# fl_sik_df.to_csv(analysis_path + "file_loadseq_seqid_key_TRAIN.csv")


('key', 'sequence_1a_24_5', 'value', 3)
('key', 'sequence_1c_34_5_fv1b', 'value', 0)
('key', 'sequence_2c_267_8_fv1b', 'value', 13)
('key', 'sequence_2b_174_5_fv1b', 'value', 7)
('key', 'sequence_1c_20_1_fv1b', 'value', 8)
('key', 'sequence_1a_64_5', 'value', 2)
('key', 'sequence_1a_57_8', 'value', 1)
('key', 'sequence_2b_187_8_fv1b', 'value', 5)
('key', 'sequence_1a_30_1', 'value', 8)
('key', 'sequence_1d_151_2', 'value', 4)
('key', 'sequence_2b_215_6_fv1b', 'value', 0)
('key', 'sequence_1a_5_6', 'value', 7)
('key', 'sequence_1a_55_6', 'value', 1)
('key', 'sequence_1b_67_8_fv1b', 'value', 5)
('key', 'sequence_2c_247_8_fv1b', 'value', 10)
('key', 'sequence_2b_186_7_fv1b', 'value', 5)
('key', 'sequence_1b_105_6_fv1b', 'value', 1)
('key', 'sequence_2b_224_5_fv1b', 'value', 1)
('key', 'sequence_1d_91_2', 'value', 5)
('key', 'sequence_1b_70_1_fv1b', 'value', 3)
('key', 'sequence_1c_15_6_fv1b', 'value', 3)
('key', 'sequence_1c_47_8_fv1b', 'value', 1)
('key', 'sequence_2c_241_2_fv1b', 'value

In [7]:
#now let's play with the test set. 

i = 0 
unique_load_sequences = [] #fill these with the 4-pairs as a numpy array
unique_load_sequences_aslists = []
sequences_4pairs_list = []
sequences_not4pairs_list = []
filename_loadseq_dict_seqidkey = {} #key is the sequence ID, value is its position in the list. 
four_pairs_condition = False
print("test_set size: ",len(test_set_filenames))
for files in test_set_filenames:
    #print("filename", files)
    i += 1
    data_load_path = test_path + '/data/' + files[0]
    label_load_path = test_path + '/label/' + files[1]
    # print("data/label load path: {} \n {}".format(data_load_path,label_load_path))
    test_array = np.load(data_load_path)
    test_label_array = np.load(label_load_path)[:, 1:]
    
    #check if the repeating unit is actually 4 tuples. 
    #four_pair_condition = (test_array[0:5,-2] == test_array[4:8,-2]) and (test_array[0:5,-1] == test_array[4:8,-1]) error. 
    #numpy version 
    four_pair_condition = np.array_equal(test_array[0:4,-2:],test_array[4:8,-2:]) 

    
    if four_pair_condition == True:
        #print(str(files[0])," is in.")
        load_sequence = test_array[0:4,-2:]
        #if len(unique_load_sequences) != 0 :
        array_equality = False
        if len(unique_load_sequences) == 0:
            unique_load_sequences.append(load_sequence)
            #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence)) #need to be flattened into a list to make it hashable for the list-operators to work
            unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        if len(unique_load_sequences) != 0:
            for load_seq in unique_load_sequences:
                if np.array_equal(load_seq,load_sequence) == True:
                    #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                    array_equality = True
            if array_equality == False:
                unique_load_sequences.append(load_sequence) #the load types
                #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
                unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
            filename_loadseq_dict_seqidkey[str(files[0])[:-4]] = unique_load_sequences_aslists.index(str(np.ndarray.flatten(load_sequence)))
        sequences_4pairs_list.append(str(files[0]))
    if four_pair_condition == False:
        sequences_not4pairs_list.append(str(files[0]))
        
    
    #print("test_array's last two columns",test_array[0:20,-2:])
#unique_load_sequences = set(unique_load_sequences)
#print("there are", len(unique_load_sequences), " load sequences")
#print("the unique load sequences are:", unique_load_sequences)
print("sequences with 4-pair repeating units: ",len(sequences_4pairs_list))
print("sequences WITHOUT 4-pair repeating units: ",sequences_not4pairs_list)

twelve_pair_condition = np.array_equal(test_array[0:12,-2:],test_array[12:24,-2:]) 
sequences_12pairs_list = []
sequences_not4_not12_pairs_list = []

#gotta run the 4-pair filter first to make sure the 12-pair subset list is ok. Hence the separate loop. 
for unusual_seqs in sequences_not4pairs_list: #12 repeating units. 
    data_load_path = test_path + '/data/' + unusual_seqs
    test_array = np.load(data_load_path)
    
    if twelve_pair_condition == True:
        #print(str(unusual_seqs)," is in.")
        load_sequence = test_array[0:12,-2:]
        #if len(unique_load_sequences) != 0 :
        array_equality = False
        if len(unique_load_sequences) == 0:
            unique_load_sequences.append(load_sequence)
            #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
            unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        if len(unique_load_sequences) != 0:
            for load_seq in unique_load_sequences:
                if np.array_equal(load_seq,load_sequence) == True:
                    #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                    array_equality = True
            if array_equality == False:
                unique_load_sequences.append(load_sequence) #the load types
                #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
                unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        sequences_12pairs_list.append(str(unusual_seqs))
        filename_loadseq_dict_seqidkey[str(unusual_seqs)[:-4]] = unique_load_sequences_aslists.index(str(np.ndarray.flatten(load_sequence)))
    if twelve_pair_condition == False:
        sequences_not4_not12_pairs_list.append(str(unusual_seqs))

print("There are ",len(sequences_12pairs_list), " sequences with 12-pairs")        
print("There are ", len(sequences_not4_not12_pairs_list), "sequences that have neither 4 nor 12 repeating units")
print("seqs that aren't 4 or 12: ",sequences_not4_not12_pairs_list)
print("there are ",len(unique_load_sequences), " different load sequences in the ", len(test_set_filenames), "sequences in the training set")
#print("unique load seqs", unique_load_sequences)

for key,value in filename_loadseq_dict_seqidkey.iteritems():
    print("key",key,"value",value)
    load_sequence_aslist = str(unique_load_sequences_aslists[value])
    #this arrangement below is necessary because dict has to have a list as its value. 
    #casting the list-cast ndarray as a string means.. parsing it back to numpy requires some creativity. 
    if filename_loadseq_dict_loadlistposkey.get(str(load_sequence_aslist)) != None:
        filename_loadseq_dict_loadlistposkey[str(load_sequence_aslist)].append(key)
    if filename_loadseq_dict_loadlistposkey.get(str(load_sequence_aslist)) == None: 
        content_list = []
        content_list.append(key)
        filename_loadseq_dict_loadlistposkey[str(load_sequence_aslist)] = content_list

print(filename_loadseq_dict_loadlistposkey)
fl_llpk_test_df = pd.DataFrame.from_dict(filename_loadseq_dict_loadlistposkey,orient='index')
fl_sik_test_df = pd.DataFrame.from_dict(filename_loadseq_dict_seqidkey,orient='index')

fl_llpk_test_df.to_csv(analysis_path + "file_loadseq_loadlistkey_TEST.csv")
fl_sik_test_df.to_csv(analysis_path + "file_loadseq_seqid_key_TEST.csv")

#append the string of the loadseq #file_loadseq_seqid_key_TEST.csv
loadseq_loadlist_df = pd.DataFrame(unique_load_sequences_aslists,columns=['loadseq'])
loadseq_loadlist_df.to_csv(analysis_path + "load_list_code.csv")
# for item in loadseq_loadlist_df:
#     loadseq_loadlist_df.loc[]


('test_set size: ', 110)
('sequences with 4-pair repeating units: ', 80)
('sequences WITHOUT 4-pair repeating units: ', ['sequence_1k_189_10.npy', 'sequence_1l_9_10.npy', 'sequence_1l_69_10.npy', 'sequence_1k_208_9.npy', 'sequence_1l_8_9.npy', 'sequence_1k_229_10.npy', 'sequence_1l_79_10.npy', 'sequence_1l_18_9.npy', 'sequence_1k_179_10.npy', 'sequence_1l_19_10.npy', 'sequence_1k_188_9.npy', 'sequence_1k_239_10.npy', 'sequence_1k_178_9.npy', 'sequence_1l_29_10.npy', 'sequence_1k_228_9.npy', 'sequence_1l_49_10.npy', 'sequence_1l_38_9.npy', 'sequence_1l_78_9.npy', 'sequence_1l_48_9.npy', 'sequence_1k_238_9.npy', 'sequence_1k_209_10.npy', 'sequence_1l_59_10.npy', 'sequence_1l_28_9.npy', 'sequence_1l_58_9.npy', 'sequence_1k_168_9.npy', 'sequence_1k_219_10.npy', 'sequence_1k_218_9.npy', 'sequence_1l_39_10.npy', 'sequence_1k_169_10.npy', 'sequence_1l_68_9.npy'])
('There are ', 30, ' sequences with 12-pairs')
('There are ', 0, 'sequences that have neither 4 nor 12 repeating units')
("seqs tha

In [8]:
#run the train set again to make the combined (test+train) dict; most of the lists aren't reset. 

i = 0 
# unique_load_sequences = [] #fill these with the 4-pairs as a numpy array
# unique_load_sequences_aslists = []
sequences_4pairs_list = []
sequences_not4pairs_list = []
# filename_loadseq_dict_seqidkey = {} #key is the sequence ID, value is its position in the list. 
four_pairs_condition = False
print("train_set size: ",len(train_set_filenames))
for files in train_set_filenames:
    #print("filename", files)
    i += 1
    data_load_path = train_path + '/data/' + files[0]
    label_load_path = train_path + '/label/' + files[1]
    # print("data/label load path: {} \n {}".format(data_load_path,label_load_path))
    train_array = np.load(data_load_path)
    train_label_array = np.load(label_load_path)[:, 1:]
    
    #check if the repeating unit is actually 4 tuples. 
    #four_pair_condition = (train_array[0:5,-2] == train_array[4:8,-2]) and (train_array[0:5,-1] == train_array[4:8,-1]) error. 
    #numpy version 
    four_pair_condition = np.array_equal(train_array[0:4,-2:],train_array[4:8,-2:]) 

    
    if four_pair_condition == True:
        #print(str(files[0])," is in.")
        load_sequence = train_array[0:4,-2:]
        #if len(unique_load_sequences) != 0 :
        array_equality = False
        if len(unique_load_sequences) == 0:
            unique_load_sequences.append(load_sequence)
            #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence)) #need to be flattened into a list to make it hashable for the list-operators to work
            unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        if len(unique_load_sequences) != 0:
            for load_seq in unique_load_sequences:
                if np.array_equal(load_seq,load_sequence) == True:
                    #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                    array_equality = True
            if array_equality == False:
                unique_load_sequences.append(load_sequence) #the load types
                #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
                unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
            filename_loadseq_dict_seqidkey[str(files[0])] = unique_load_sequences_aslists.index(str(np.ndarray.flatten(load_sequence)))
        sequences_4pairs_list.append(str(files[0]))
    if four_pair_condition == False:
        sequences_not4pairs_list.append(str(files[0]))
        
    
    #print("train_array's last two columns",train_array[0:20,-2:])
#unique_load_sequences = set(unique_load_sequences)
#print("there are", len(unique_load_sequences), " load sequences")
#print("the unique load sequences are:", unique_load_sequences)
print("sequences with 4-pair repeating units: ",len(sequences_4pairs_list))
print("sequences WITHOUT 4-pair repeating units: ",sequences_not4pairs_list)

twelve_pair_condition = np.array_equal(train_array[0:12,-2:],train_array[12:24,-2:]) 
sequences_12pairs_list = []
sequences_not4_not12_pairs_list = []

#gotta run the 4-pair filter first to make sure the 12-pair subset list is ok. Hence the separate loop. 
for unusual_seqs in sequences_not4pairs_list: #12 repeating units. 
    data_load_path = train_path + '/data/' + unusual_seqs
    train_array = np.load(data_load_path)
    
    if twelve_pair_condition == True:
        #print(str(unusual_seqs)," is in.")
        load_sequence = train_array[0:12,-2:]
        #if len(unique_load_sequences) != 0 :
        array_equality = False
        if len(unique_load_sequences) == 0:
            unique_load_sequences.append(load_sequence)
            #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
            unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        if len(unique_load_sequences) != 0:
            for load_seq in unique_load_sequences:
                if np.array_equal(load_seq,load_sequence) == True:
                    #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                    array_equality = True
            if array_equality == False:
                unique_load_sequences.append(load_sequence) #the load types
                #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
                unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        sequences_12pairs_list.append(str(unusual_seqs))
        filename_loadseq_dict_seqidkey[str(unusual_seqs)] = unique_load_sequences_aslists.index(str(np.ndarray.flatten(load_sequence)))
    if twelve_pair_condition == False:
        sequences_not4_not12_pairs_list.append(str(unusual_seqs))

print("There are ",len(sequences_12pairs_list), " sequences with 12-pairs")        
print("There are ", len(sequences_not4_not12_pairs_list), "sequences that have neither 4 nor 12 repeating units")
print("seqs that aren't 4 or 12: ",sequences_not4_not12_pairs_list)
print("there are ",len(unique_load_sequences), " different load sequences in the ", len(train_set_filenames), "sequences in the training set")
#print("unique load seqs", unique_load_sequences)

for key,value in filename_loadseq_dict_seqidkey.iteritems():
    print("key",key,"value",value)
    load_sequence_aslist = str(unique_load_sequences_aslists[value])
    #this arrangement below is necessary because dict has to have a list as its value. 
    #casting the list-cast ndarray as a string means.. parsing it back to numpy requires some creativity. 
    if filename_loadseq_dict_loadlistposkey.get(str(load_sequence_aslist)) != None:
        filename_loadseq_dict_loadlistposkey[str(load_sequence_aslist)].append(key)
    if filename_loadseq_dict_loadlistposkey.get(str(load_sequence_aslist)) == None: 
        content_list = []
        content_list.append(key)
        filename_loadseq_dict_loadlistposkey[str(load_sequence_aslist)] = content_list

print(filename_loadseq_dict_loadlistposkey)
fl_llpk_combined_df = pd.DataFrame.from_dict(filename_loadseq_dict_loadlistposkey,orient='index')
fl_sik_combined_df = pd.DataFrame.from_dict(filename_loadseq_dict_seqidkey,orient='index')
fl_llpk_combined_df.to_csv(analysis_path + "file_loadseq_loadlistkey_COMBINED.csv")
fl_sik_combined_df.to_csv(analysis_path + "file_loadseq_seqid_key_COMBINED.csv")


('train_set size: ', 440)
('sequences with 4-pair repeating units: ', 320)
('sequences WITHOUT 4-pair repeating units: ', ['sequence_1l_63_4.npy', 'sequence_1l_31_2.npy', 'sequence_1l_37_8.npy', 'sequence_1k_207_8.npy', 'sequence_1k_176_7.npy', 'sequence_1k_161_2.npy', 'sequence_1l_7_8.npy', 'sequence_1l_52_3.npy', 'sequence_1l_65_6.npy', 'sequence_1l_30_1.npy', 'sequence_1l_66_7.npy', 'sequence_1k_201_2.npy', 'sequence_1k_215_6.npy', 'sequence_1k_235_6.npy', 'sequence_1l_4_5.npy', 'sequence_1k_212_3.npy', 'sequence_1k_210_1.npy', 'sequence_1k_175_6.npy', 'sequence_1k_170_1.npy', 'sequence_1k_163_4.npy', 'sequence_1k_162_3.npy', 'sequence_1l_76_7.npy', 'sequence_1l_62_3.npy', 'sequence_1l_53_4.npy', 'sequence_1k_186_7.npy', 'sequence_1k_166_7.npy', 'sequence_1l_35_6.npy', 'sequence_1l_51_2.npy', 'sequence_1k_222_3.npy', 'sequence_1l_12_3.npy', 'sequence_1k_223_4.npy', 'sequence_1k_234_5.npy', 'sequence_1k_225_6.npy', 'sequence_1l_3_4.npy', 'sequence_1l_44_5.npy', 'sequence_1l_26_7.npy'

('key', 'sequence_1k_239_10', 'value', 22)
('key', 'sequence_2c_237_8_fv1b.npy', 'value', 11)
('key', 'sequence_2c_261_2_fv1b.npy', 'value', 6)
('key', 'sequence_2b_194_5_fv1b.npy', 'value', 5)
('key', 'sequence_1c_8_9_fv1b', 'value', 3)
('key', 'sequence_1c_7_8_fv1b.npy', 'value', 3)
('key', 'sequence_2c_289_10_fv1b', 'value', 1)
('key', 'sequence_1a_12_3.npy', 'value', 3)
('key', 'sequence_1a_3_4.npy', 'value', 4)
('key', 'sequence_1c_34_5_fv1b.npy', 'value', 7)
('key', 'sequence_2b_228_9_fv1b', 'value', 2)
('key', 'sequence_1k_209_10', 'value', 17)
('key', 'sequence_1c_26_7_fv1b.npy', 'value', 9)
('key', 'sequence_1l_58_9', 'value', 26)
('key', 'sequence_1d_92_3.npy', 'value', 3)
('key', 'sequence_1l_78_9', 'value', 19)
('key', 'sequence_1d_140_1.npy', 'value', 0)
('key', 'sequence_1c_18_9_fv1b', 'value', 5)
('key', 'sequence_1a_36_7.npy', 'value', 9)
('key', 'sequence_1b_83_4_fv1b.npy', 'value', 9)
('key', 'sequence_1a_1_2.npy', 'value', 4)
('key', 'sequence_1d_134_5.npy', 'value',

In [9]:
print(len(unique_load_sequences))
print(unique_load_sequences[0].shape)
for item in unique_load_sequences:
    print(item.shape)

29
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)


In [10]:
flattened_loadseq_list = []
for item in unique_load_sequences:
    flattened_loadseq_list.append(np.ndarray.flatten(item))
print(flattened_loadseq_list[0].shape,flattened_loadseq_list[-1].shape)

((8,), (24,))


In [11]:
sigma1 = flattened_loadseq_list[0]
sigma2 = flattened_loadseq_list[-1]
print(np.mean(sigma1), np.median(sigma1), np.std(sigma1))
print(flattened_loadseq_list[0])

(7.8540000000000001, 7.8540000000000001, 6.4260000000000002)
[  1.428  14.28   14.28    1.428   1.428  14.28   14.28    1.428]


In [12]:
#flattened_loadseq_list.sort() #doesn't work
#try to pad
flattened_eq_len_loadseq_list = []
long_shape = flattened_loadseq_list[-1].shape
for item in flattened_loadseq_list:
    if item.shape == (8,):
        temp_padded_array = np.zeros(shape=long_shape)
        temp_padded_array[0:8]=item
        temp_padded_array[8:16]=item
        temp_padded_array[16:24]=item
        flattened_eq_len_loadseq_list.append(temp_padded_array)
    if item.shape == (24,):
        flattened_eq_len_loadseq_list.append(item)

# for item in flattened_eq_len_loadseq_list:
#     print(item.shape)
any(item.shape != (24,) for item in flattened_eq_len_loadseq_list)

False

In [13]:
print(str(flattened_eq_len_loadseq_list[0])) # you can cast this as a string. yay. 

[  1.428  14.28   14.28    1.428   1.428  14.28   14.28    1.428   1.428
  14.28   14.28    1.428   1.428  14.28   14.28    1.428   1.428  14.28
  14.28    1.428   1.428  14.28   14.28    1.428]


In [14]:
posn=0
for item in flattened_eq_len_loadseq_list:
    print(posn,np.mean(item), np.median(item), np.std(item))
    posn+=1

(0, 7.8539999999999992, 7.8540000000000001, 6.4260000000000002)
(1, 7.9040000000000008, 6.6539999999999999, 7.0996039326148335)
(2, 7.8539999999999992, 6.8540000000000001, 6.5797778077986795)
(3, 7.8094999999999999, 7.8540000000000001, 6.4708060355723847)
(4, 7.7469999999999999, 7.8540000000000001, 6.5347522523811099)
(5, 7.9344999999999999, 8.0150000000000006, 6.3465211533563801)
(6, 5.5844999999999994, 5.6539999999999999, 4.6965285850295855)
(7, 8.1844999999999999, 7.8540000000000001, 7.1346745370759557)
(8, 8.0845000000000002, 7.6539999999999999, 7.230828496790668)
(9, 7.6844999999999999, 7.8540000000000001, 6.5998546006711383)
(10, 7.0844999999999994, 6.0149999999999997, 5.9689932777646844)
(11, 5.8969999999999994, 5.6539999999999999, 4.8957927856476937)
(12, 8.1219999999999999, 7.8540000000000001, 7.2003202706546316)
(13, 7.2094999999999994, 7.2649999999999997, 6.0710073916937377)
(14, 7.8511666666666668, 8.0150000000000006, 6.4331539417993788)
(15, 7.8019999999999996, 8.0, 5.1613

In [15]:
print(flattened_eq_len_loadseq_list[21])

[  1.428  14.28   14.28    1.25    1.25   14.28   14.28    1.428   1.428
  14.28   14.28    1.75    1.75   14.28   14.28    1.428   1.428  14.28
  14.28    1.25    1.25   14.28   14.28    1.428]


In [16]:
#create a dataframe that correlates each sequence (in both sets) to the equal-lengthened array
#fl_llpk_combined_df = pd.DataFrame.from_dict(filename_loadseq_dict_loadlistposkey,orient='index')
#fl_sik_combined_df = pd.DataFrame.from_dict(filename_loadseq_dict_seqidkey,orient='index')
filename_loadseq_dict_seqidkey_equallength = {}
print(filename_loadseq_dict_seqidkey.keys())
print(len(filename_loadseq_dict_seqidkey.keys()))

['sequence_2c_232_3_fv1b.npy', 'sequence_1c_38_9_fv1b', 'sequence_1k_179_10', 'sequence_2b_198_9_fv1b', 'sequence_1b_97_8_fv1b.npy', 'sequence_1a_40_1.npy', 'sequence_1a_20_1.npy', 'sequence_1c_47_8_fv1b.npy', 'sequence_1a_75_6.npy', 'sequence_1a_16_7.npy', 'sequence_1d_124_5.npy', 'sequence_1d_82_3.npy', 'sequence_2c_270_1_fv1b.npy', 'sequence_1d_158_9', 'sequence_1a_23_4.npy', 'sequence_1l_48_9', 'sequence_1b_88_9_fv1b', 'sequence_1b_66_7_fv1b.npy', 'sequence_1k_191_2.npy', 'sequence_1a_5_6.npy', 'sequence_1a_30_1.npy', 'sequence_1d_130_1.npy', 'sequence_1k_238_9', 'sequence_1d_81_2.npy', 'sequence_1b_50_1_fv1b.npy', 'sequence_1b_79_10_fv1b', 'sequence_1b_72_3_fv1b.npy', 'sequence_1b_67_8_fv1b.npy', 'sequence_1d_145_6.npy', 'sequence_2c_240_1_fv1b.npy', 'sequence_1a_68_9', 'sequence_1d_94_5.npy', 'sequence_1l_79_10', 'sequence_2c_275_6_fv1b.npy', 'sequence_1d_131_2.npy', 'sequence_2c_254_5_fv1b.npy', 'sequence_1b_63_4_fv1b.npy', 'sequence_1b_57_8_fv1b.npy', 'sequence_1l_49_10', 'sequ

In [17]:
# for key, value in filename_loadseq_dict_seqidkey:
#     filename_loadseq_dict_seqidkey_equallength[key] = 
loadseq_df = pd.DataFrame(flattened_loadseq_list)
loadseq_df.to_csv(analysis_path+"loadseq_flattened.csv")

In [18]:
#fl_sik_test_df file-load sequenceid-key-TEST
#loadseq_loadlist_df
fl_sik_test_df.index.rename(name='seq_name',inplace=True)
fl_sik_test_df.columns = ['loadseq_id']

loadseq_loadlist_df.index.rename(name='loadseq_id',inplace=True)
loadseq_loadlist_df.columns = ['loadseq']


print(fl_sik_test_df.columns,fl_sik_test_df.index)
print(loadseq_loadlist_df.columns,loadseq_loadlist_df.index)
complete_seqname_loadseqlist_df = pd.merge(fl_sik_test_df,loadseq_loadlist_df,how='left',right_index=True,left_on='loadseq_id')
print(complete_seqname_loadseqlist_df.head(2))

(Index([u'loadseq_id'], dtype='object'), Index([u'sequence_2c_239_10_fv1b', u'sequence_1a_78_9',
       u'sequence_1c_9_10_fv1b', u'sequence_1c_38_9_fv1b',
       u'sequence_2c_289_10_fv1b', u'sequence_2b_189_10_fv1b',
       u'sequence_2b_199_10_fv1b', u'sequence_1k_179_10',
       u'sequence_2b_198_9_fv1b', u'sequence_1b_69_10_fv1b',
       ...
       u'sequence_1k_218_9', u'sequence_1l_69_10', u'sequence_1c_29_10_fv1b',
       u'sequence_1k_169_10', u'sequence_2b_218_9_fv1b',
       u'sequence_1b_109_10_fv1b', u'sequence_1b_89_10_fv1b',
       u'sequence_1c_48_9_fv1b', u'sequence_1d_138_9',
       u'sequence_2c_269_10_fv1b'],
      dtype='object', name=u'seq_name', length=110))
(Index([u'loadseq'], dtype='object'), RangeIndex(start=0, stop=29, step=1, name=u'loadseq_id'))
                         loadseq_id  \
seq_name                              
sequence_2c_239_10_fv1b          11   
sequence_1a_78_9                 12   

                                                         

In [19]:
complete_seqname_loadseqlist_df.to_csv(analysis_path+"complete_seqname_loadseqlist.csv")